# MobiML GeoTrackNet Demo

Based on: https://github.com/CIA-Oceanix/GeoTrackNet (MIT Licensed, (c) 2018 Duong Nguyen)

As presented in Nguyen, D., Vadaine, R., Hajduch, G., Garello, R. (2022). GeoTrackNet - A Maritime Anomaly Detector Using Probabilistic Neural Network Representation of AIS Tracks and A Contrario Detection. In IEEE Transactions on Intelligent Transportation Systems, 23(6).


Using data from AISDK: http://web.ais.dk/aisdata/aisdk-2018-02.zip

*It is possible to further explore maritime traffic patterns with the TrAISformer (https://github.com/CIA-Oceanix/TrAISformer), which is used for vessel trajectory prediction. The TrAISformer can be trained with AIS data and the preprocessing steps are similar to those of GeoTrackNet. However, the TrAISformer is out of the scope of MobiML and is an optional extension for the user to explore.*

## Environments

### Preprocessing

It is recommended to perform the preprocessing steps with the MobiML environment.

### Model Training

Set up a dedicated GeoTrackNet environment (PY3GPU) to train the model as instructed by Nguyen et al. (2022).

## Preprocessing

In [ ]:
import numpy as np
import os
import sys
import pickle
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from datetime import datetime, timedelta

sys.path.append("..")
from mobiml.datasets import AISDK
from mobiml.preprocessing import (
    TrajectorySplitter,
    TrajectoryFilter,
    TrajectorySubsampler,
)
from mobiml.transforms import TemporalSplitter

In [ ]:
%pwd

In [3]:
# AISDK dataset
LAT, LON, SOG, COG, NAME, SHIPTYPE, NAV_STT, TIMESTAMP, TRAJ_ID = list(range(9))

EPOCH = datetime(1970, 1, 1)

SOG_MIN = 2.0
SOG_MAX = 30.0  # SOG is truncated to 30.0 knots max

# Pkl filenames
pkl_filename_train = "aisdk_20180208_train.pkl"
pkl_filename_valid = "aisdk_20180208_valid.pkl"
pkl_filename_test = "aisdk_20180208_test.pkl"

# Path to csv files
data_path = "data/aisdk_20180208_sample/"
csv_filename = "aisdk_20180208_sample.csv"

# Output path
out_path = "data/aisdk_20180208_sample/"

##### Set up bounding box

### Loading data

In [ ]:
path = os.path.join(data_path, csv_filename)
print(f"{datetime.now()} Loading data from {path}")
aisdk = AISDK(path)  # you can specify a bounding box here to filter the area
LON_MIN, LAT_MIN, LON_MAX, LAT_MAX = aisdk.get_bounds()
print(
    f"Bounding box:\nmin_lon: {LON_MIN}\nmin_lat: {LAT_MIN}\nmax_lon: {LON_MAX}\nmax_lat: {LAT_MAX}"
)

### Cleaning

#### Remove missing values

In [ ]:
aisdk.df = aisdk.df.dropna()
aisdk.df

In [ ]:
print("After removing missing values we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])
print("Total number of vessels:", len(aisdk.df.traj_id.unique()))
print("Lat min: ", aisdk.df.y.min(), "Lat max: ", aisdk.df.y.max())
print("Lon min: ", aisdk.df.x.min(), "Lon max: ", aisdk.df.x.max())
print("Time min: ", aisdk.df.timestamp.min(), "Time max: ", aisdk.df.timestamp.max())

#### Remove 'Moored' and 'At anchor' AIS messages

In [ ]:
aisdk.df.drop(aisdk.df[(aisdk.df["nav_status"] == "Moored") | (aisdk.df["nav_status"] == "At anchor")].index, inplace=True)
print("After removing 'Moored' or 'At anchor' AIS messages we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Keep only 'Cargo', 'Tanker', 'Passenger' vessel types

In [ ]:
aisdk.df = aisdk.df[
    (aisdk.df["ship_type"] == "Cargo")
    | (aisdk.df["ship_type"] == "Tanker")
    | (aisdk.df["ship_type"] == "Passenger")
]
print("After keeping only 'Cargo', 'Tanker' or 'Passenger' AIS messages we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Split trajectories with observation gaps > 2 hrs

In [ ]:
aisdk = TrajectorySplitter(aisdk).split(observation_gap=timedelta(hours=2))
print("After splitting trajectories with observation gaps we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Drop trajectories with fewer than $Points_{min}$ locations

In [ ]:
aisdk = TrajectoryFilter(aisdk).filter_min_pts(min_pts=20)
print("After removing trajectories with too few points we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Drop speed outliers

In [ ]:
aisdk = TrajectoryFilter(aisdk).filter_speed(min_speed=SOG_MIN, max_speed=SOG_MAX)
print("After removing speed outliers by setting a minimum and maximum speed we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

In [23]:
tc = aisdk.to_trajs() #  mpd.TrajectoryCollection(aisdk.df, "traj_id", t="timestamp", x="x", y="y")
traj_gdf = tc.to_traj_gdf()

We may also want to remove trajectories based on their overall average speed rather than the SOG values:

In [28]:
for index, row in traj_gdf.iterrows():
    traj_gdf.loc[index, "speed_ok"] = (
        tc.trajectories[index].get_length()
        / tc.trajectories[index].get_duration().total_seconds()
        > 1.02889  # 2 knots
    )

In [29]:
traj_gdf = traj_gdf[traj_gdf["speed_ok"] == True]

In [ ]:
aisdk.df = pd.merge(aisdk.df, traj_gdf["traj_id"], how="inner")
print("After removing speed outliers based on length and duration we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

### Training data preparation
#### Subsample AIS tracks 

In [ ]:
aisdk = TrajectorySubsampler(aisdk).subsample(min_dt_sec=60)
print("After subsampling AIS tracks we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Temporal train/valid/test split

In [ ]:
aisdk = TemporalSplitter(aisdk).split_hr()
aisdk.df

In [ ]:
aisdk_train = aisdk.df[(aisdk.df["split"] == 1.0)]
aisdk_valid = aisdk.df[(aisdk.df["split"] == 2.0)]
aisdk_test = aisdk.df[(aisdk.df["split"] == 3.0)]

print("Total number of AIS messages: ", len(aisdk.df))
print("Number of msgs in the training set: ", len(aisdk_train))
print("Number of msgs in the validation set: ", len(aisdk_valid))
print("Number of msgs in the test set: ", len(aisdk_test))

In [ ]:
aisdk_train

In [ ]:
target_column_order=["y", "x", "speed", "direction", "Name", "ship_type", "nav_status", "timestamp", "traj_id"]
aisdk_train = aisdk_train[target_column_order].reset_index(drop=True)
aisdk_valid = aisdk_valid[target_column_order].reset_index(drop=True)
aisdk_test = aisdk_test[target_column_order].reset_index(drop=True)
aisdk_test

#### Format timestamp

In [ ]:
aisdk_train["timestamp"] = (aisdk_train["timestamp"].astype(int) / 1_000_000_000).astype(int)
aisdk_valid["timestamp"] = (aisdk_valid["timestamp"].astype(int) / 1_000_000_000).astype(int)
aisdk_test["timestamp"]  = (aisdk_test["timestamp"].astype(int) / 1_000_000_000).astype(int)
aisdk_test

#### Format to ndarrays

In [52]:
aisdk_train = np.array(aisdk_train)
aisdk_valid = np.array(aisdk_valid)
aisdk_test = np.array(aisdk_test)

#### Merging into dict
Creating AIS tracks from the list of AIS messages. Each AIS track is formatted by a dictionary.

In [ ]:
print("Convert to dicts of vessel's tracks...")

def convert_tracks_to_dicts(tracks):
    d = dict()
    for v_msg in tqdm(tracks):
        mmsi = int(v_msg[TRAJ_ID])
        if not (mmsi in list(d.keys())):
            d[mmsi] = np.empty((0, 9))
        d[mmsi] = np.concatenate(
            (d[mmsi], np.expand_dims(v_msg[:9], 0)), axis=0
        )
    for key in tqdm(list(d.keys())):
        d[key] = np.array(
            sorted(d[key], key=lambda m_entry: m_entry[TIMESTAMP])
        )
    return d

Vs_train = convert_tracks_to_dicts(aisdk_train)
Vs_valid = convert_tracks_to_dicts(aisdk_valid)
Vs_test = convert_tracks_to_dicts(aisdk_test)

#### Normalisation

In [ ]:
print("Normalising data ...")

def normalize(d):
    for k in tqdm(list(d.keys())):
        v = d[k]
        v[:, LAT] = (v[:, LAT] - LAT_MIN) / (LAT_MAX - LAT_MIN)
        v[:, LON] = (v[:, LON] - LON_MIN) / (LON_MAX - LON_MIN)
        v[:, SOG][v[:, SOG] > SOG_MAX] = SOG_MAX
        v[:, SOG] = v[:, SOG] / SOG_MAX
        v[:, COG] = v[:, COG] / 360.0
    return d 

Vs_train = normalize(Vs_train)
Vs_valid = normalize(Vs_valid)
Vs_test = normalize(Vs_test)

In [ ]:
for filename, filedict in zip(
    [pkl_filename_train, pkl_filename_valid, pkl_filename_test],
    [Vs_train, Vs_valid, Vs_test],
):
    print("Writing to", os.path.join(out_path, filename))
    with open(os.path.join(out_path, filename), "wb") as f:
        pickle.dump(filedict, f)

## Model Training

From this point forward, it is recommended to execute the code with the [PY3GPU environment](https://github.com/CIA-Oceanix/GeoTrackNet/blob/master/requirements.yml), as set up by Nguyen et al. (2022).

### Setup

In [1]:
# AISDK dataset
LAT, LON, SOG, COG, NAME, SHIPTYPE, NAV_STT, TIMESTAMP, TRAJ_ID = list(range(9))

# Pkl filenames
pkl_filename_train = "aisdk_20180208_train.pkl"
pkl_filename_valid = "aisdk_20180208_valid.pkl"
pkl_filename_test = "aisdk_20180208_test.pkl"

# Path to csv files
data_path = "../examples/data/aisdk_20180208_sample/"
csv_filename = "aisdk_20180208_sample_20000.csv"

# Output path
out_path = "../examples/data/aisdk_20180208_sample/"

### Calculate AIS mean

In [ ]:
"""
Input pipelines script for Tensorflow graph.
This script is adapted from the original script of FIVO.
"""

import numpy as np
import pickle
import os
import tensorflow as tf

dataset_path = os.path.join(data_path, pkl_filename_train)

LAT_BINS = 100
LON_BINS = 200
SOG_BINS = 30
COG_BINS = 72


def sparse_AIS_to_dense(msgs_, num_timesteps, mmsis):
    def create_dense_vect(msg, lat_bins=100, lon_bins=200, sog_bins=30, cog_bins=72):
        lat, lon, sog, cog = msg[0], msg[1], msg[2], msg[3]
        data_dim = lat_bins + lon_bins + sog_bins + cog_bins
        dense_vect = np.zeros(data_dim)
        dense_vect[int(lat * lat_bins)] = 1.0
        dense_vect[int(lon * lon_bins) + lat_bins] = 1.0
        dense_vect[int(sog * sog_bins) + lat_bins + lon_bins] = 1.0
        dense_vect[int(cog * cog_bins) + lat_bins + lon_bins + sog_bins] = 1.0
        return dense_vect

    dense_msgs = []
    for msg in msgs_:
        dense_msgs.append(
            create_dense_vect(
                msg,
                lat_bins=LAT_BINS,
                lon_bins=LON_BINS,
                sog_bins=SOG_BINS,
                cog_bins=COG_BINS,
            )
        )
    dense_msgs = np.array(dense_msgs)
    return dense_msgs, num_timesteps, mmsis


dirname = os.path.dirname(dataset_path)

try:
    with tf.gfile.Open(dataset_path, "rb") as f:
        Vs = pickle.load(f)
except:
    with tf.gfile.Open(dataset_path, "rb") as f:
        Vs = pickle.load(f, encoding="latin1")

data_dim = LAT_BINS + LON_BINS + SOG_BINS + COG_BINS

mean_all = np.zeros((data_dim,))
sum_all = np.zeros((data_dim,))
total_ais_msg = 0

current_mean = np.zeros((0, data_dim))
current_ais_msg = 0

count = 0
for mmsi in list(Vs.keys()):
    count += 1
    print(count)
    tmp = Vs[mmsi][:, [LAT, LON, SOG, COG]]
    tmp[tmp == 1] = 0.99999
    current_sparse_matrix, _, _ = sparse_AIS_to_dense(tmp, 0, 0)
    #    current_mean = np.mean(current_sparse_matrix,axis = 0)
    sum_all += np.sum(current_sparse_matrix, axis=0)
    total_ais_msg += len(current_sparse_matrix)

mean = sum_all / total_ais_msg

print("Writing to", os.path.join(dirname, "/mean.pkl"))
with open(dirname + "/mean.pkl", "wb") as f:
    pickle.dump(mean, f)

### Training

In [ ]:
mode = "train"
dataset_dir = "../examples/data/aisdk_20180208_sample"
trainingset_name = "aisdk_20180208_train.pkl"
testset_name = "aisdk_20180208_test.pkl"
lat_min = 57.4
lat_max = 57.9
lon_min = 11.3
lon_max = 12.1
latent_size = 100
batch_size = 32
num_samples = 16
learning_rate = 0.0003
%run -i "../mobiml/models/geotracknet.py"

In [ ]:
# RUN TRAIN
# ======================================
from mobiml.models.flags_config import config

if config.mode == "train":
    print(config.trainingset_path)
    fh = logging.FileHandler(os.path.join(config.logdir, config.log_filename + ".log"))
    tf.logging.set_verbosity(tf.logging.INFO)
    # get TF logger
    logger = logging.getLogger("tensorflow")
    logger.addHandler(fh)
    runners.run_train(config)

else:
    with open(config.testset_path, "rb") as f:
        Vs_test = pickle.load(f)
    dataset_size = len(Vs_test)